In [ ]:
import mlflow; print("Versión MLFLOW:", mlflow.__version__)
import torch ; print("VERSIÓN TORCH:", torch.__version__)
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import time
from huggingface_hub import login
from itertools import product
import pandas as pd ;print("VERSIÓN PANDAS:", pd.__version__)
import gc

token = "TU TOKEN DE HF"
login(token)

# Configuración de MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
#model_name = "deepseek-ai/Deepseek-R1-Distill-Qwen-1.5B"
#model_name = "ozone-research/Chirp-01"
model_name = "meta-llama/Llama-3.2-3B-Instruct"
#model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
#experimento = "granite-3.2-2b-instruct"
experimento = "Llama-3.2-3B-Instruct"
mlflow.set_experiment(experimento)
experimento_cuantizado = experimento + "-Cuantizado"

usar_quantizacion = False
usar_pipeline = True

In [ ]:
if usar_pipeline:
    if usar_quantizacion:
        print("Usando quantización 4-bit con bitsandbytes.")
        mlflow.set_experiment(experimento_cuantizado)

        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant = True,
            bnb_4bit_compute_dtype=torch.bfloat16
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            trust_remote_code=True, 
            device_map="auto",
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) 

        text_generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

    else:
        text_generator = pipeline(
            "text-generation",
            model=model_name,
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
    print(f"El modelo '{model_name}' es compatible con pipeline().")

else:
    print(f"El modelo '{model_name}' NO es compatible con pipeline(). Cargando manualmente.")

    if usar_quantizacion:
        print("🔹 Usando quantización 4-bit con bitsandbytes.")
        mlflow.set_experiment(experimento_cuantizado)

        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto"
        )
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

In [4]:
# Definición de queries
queries = {
    "en": [
        {"role": "system", "content": "You are a useful AI assistant that extracts possible words related to the user's intent. Keep in mind that the keywords are not only within the given phrase. Present all your ideas without asking me, and delve deeply into the topic. Print only a list of keywords or ideas, without introduction, conclusion, or connectors between ideas. Use a comma separated list."},
        {"role": "user", "content": "I want to search information about the wetland in Torrevieja called Parque Natural de las Lagunas de la Mata."}
    ],
    "es": [
        {'role': 'system', 'content': 'Eres un útil asistente de IA que extrae posibles palabras relacionadas con la intención del usuario. Ten en cuenta que las palabras clave no solo están en la frase. Plasma todas tus ideas sin consultarme, es decir, profundiza en el tema. Imprime únicamente una lista separada por comas con las palabras clave o ideas, nada más, sin introducción ni conclusión ni conectores entre ideas.'}, 
        {'role': 'user', 'content': 'Quiero buscar información sobre un humedal en Torrevieja llamado Parque Natural de las Lagunas de la Mata.'}
    ]
}

In [5]:
# Malla de parámetros
temperaturas = [0.4, 0.7, 1.0]
top_keys_vals = [20, 40, 60]
top_words_vals = [0.85, 0.9, 0.95]
num_beams_vals = [1, 2, 4]

# Generar todas las combinaciones posibles
param_combinations = list(product(temperaturas, top_keys_vals, top_words_vals, num_beams_vals))

In [ ]:
# Función para ejecutar el modelo y registrar en MLflow
def ejecutar_experimento(query_name, messages, temp_val, top_k_val, top_p_val, num_beams_val, usar_pipeline, usar_quantizacion):
    previous_runs = mlflow.search_runs(order_by=["start_time"])
    print(messages)

    if "tags.mlflow.runName" in previous_runs.columns:
        count = sum(previous_runs["tags.mlflow.runName"].fillna("").str.startswith(query_name)) + 1
    else:
        count = 1  
        
    run_name = f"{query_name}-{count}"  # Nombre en formato idioma-contador
    
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", run_name)
        mlflow.log_param("modelo", model_name)
        mlflow.log_param("query_idioma", query_name)
        mlflow.log_param("temperatura", temp_val)
        mlflow.log_param("top_k", top_k_val)
        mlflow.log_param("top_p", top_p_val)
        mlflow.log_param("num_beams", num_beams_val)
        mlflow.log_param("quantizado", usar_quantizacion)

        torch.cuda.reset_peak_memory_stats()
        inicio = time.time()

        if usar_pipeline:    
            output = text_generator(
                messages,
                max_new_tokens = 500,
                do_sample=True,
                temperature=temp_val,
                top_k=top_k_val,
                top_p=top_p_val,
                num_beams=num_beams_val,
                repetition_penalty=1.2,
                return_full_text=False
            )
            
            respuesta = output[0]["generated_text"]
            final = time.time() - inicio

            palabras = respuesta.split(" ")
            cantidad_palabras = len(palabras)
            
            keywords = respuesta.split(",")
            cantidad_keywords = len(keywords)
            mlflow.log_metric("palabras", cantidad_palabras)
            mlflow.log_metric("palabrasxsegundo", cantidad_palabras / final)
            mlflow.log_metric("keywords", cantidad_keywords)
            mlflow.log_metric("keywordsxsegundo", cantidad_keywords / final)
        else:
            input_ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=False,
                return_tensors="pt"
            )
    
            with torch.no_grad():
                output = model.generate(
                    input_ids.to("cuda"),
                    eos_token_id=tokenizer.eos_token_id,
                    max_new_tokens = 500,
                    do_sample=True,
                    temperature=temp_val,
                    top_k=top_k_val,
                    top_p=top_p_val,
                    num_beams=num_beams_val,
                    repetition_penalty=1.2
                )
    
            respuesta = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
            if (query_name == "en"):
                respuesta = respuesta.split("assistant")[2]
            else:
                respuesta = respuesta.split("assistant")[1]
                
            final = time.time() - inicio

            palabras = respuesta.split(" ")
            cantidad_palabras = len(palabras)
            
            keywords = respuesta.split(",")
            cantidad_keywords = len(keywords)

            mlflow.log_metric("palabras", cantidad_palabras)
            mlflow.log_metric("palabrasxsegundo", cantidad_palabras / final)
            mlflow.log_metric("keywords", cantidad_keywords)
            mlflow.log_metric("keywordsxsegundo", cantidad_keywords / final)
        
        memoria_usada = round(torch.cuda.memory_allocated() / (1024**2), 2)
        memoria_max_usada = round(torch.cuda.max_memory_allocated() / (1024**2), 2)

        mlflow.set_tag("respuesta", respuesta)
        mlflow.log_metric("tiempo_ejecucion", time.time() - inicio)
        mlflow.log_metric("gpu_mem_usada_MB", memoria_usada)
        mlflow.log_metric("gpu_mem_max_usada_MB", memoria_max_usada)
    
        print(f"\n--- Respuesta ---\n", respuesta)
        print("Tiempo de ejecución:", time.time() - inicio)

# Ejecutar experimentos en inglés y español
for idx, (temp_val, top_k_val, top_p_val, num_beams_val) in enumerate(param_combinations):
    ejecutar_experimento("en", queries["en"], temp_val, top_k_val, top_p_val, num_beams_val, usar_pipeline, usar_quantizacion)
    ejecutar_experimento("es", queries["es"], temp_val, top_k_val, top_p_val, num_beams_val, usar_pipeline, usar_quantizacion)

In [ ]:
def calcular_estadisticas_experimentos():
    runs = mlflow.search_runs(order_by=["start_time"])

    runs = runs[["experiment_id", "tags.mlflow.runName", 
                 "metrics.tiempo_ejecucion", "metrics.palabrasxsegundo", "metrics.keywordsxsegundo"]]
    
    runs["Idioma"] = runs["tags.mlflow.runName"].str[:2]  

    experiment_ids = runs["experiment_id"].unique()
    experiment_names = {exp_id: mlflow.get_experiment(exp_id).name for exp_id in experiment_ids}

    runs["Nombre Modelo"] = runs["experiment_id"].map(experiment_names)

    resumen = runs.groupby(["Nombre Modelo", "Idioma"]).agg({
        "metrics.tiempo_ejecucion": "mean",
        "metrics.palabrasxsegundo": "mean",
        "metrics.keywordsxsegundo": "mean"
    }).reset_index()

    resumen.columns = ["Modelo", "Idioma", "Media Tiempo (s)", "Media Palabras/s", "Media Keywords/s"]
    return resumen

tabla_resumen = calcular_estadisticas_experimentos()
tabla_resumen

,Modelo,Idioma,Media Tiempo (s),Media Palabras/s,Media Keywords/s
0,Llama-3.2-3B,en,11.420085,14.469464,7.973382
1,Llama-3.2-3B,es,3.275390,9.749098,7.176375


In [ ]:
def liberar_memoria_gpu():
    global model, tokenizer, text_generator  

    if 'model' in globals():
        del model
    if 'tokenizer' in globals():
        del tokenizer
    if 'text_generator' in globals():
        del text_generator

    # Forzar recolección de basura
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    print("Memoria GPU liberada correctamente.")

liberar_memoria_gpu()